In [1]:
!pip install psycopg2-binary

  Using cached psycopg2_binary-2.8.6-cp38-cp38-win_amd64.whl (1.1 MB)


In [2]:
import psycopg2

In [94]:
con = psycopg2.connect(database="postgres", user="postgres", password="postgres", host="127.0.0.1", port="5432")

In [95]:
cur = con.cursor()

In [96]:
cur.execute("select * from pubs2.authors")
rows = cur.fetchall()
for row in rows:
    print(row)

('409-56-7008', 'Bennet', 'Abraham', '415 658-9932', '6223 Bateman St.', 'Berkeley', 'CA', '94705', 1)
('213-46-8915', 'Green', 'Marjorie', '415 986-7020', '309 63rd St. #411', 'Oakland', 'CA', '94618', 1)
('238-95-7766', 'Carson', 'Cheryl', '415 548-7723', '589 Darwin Ln.', 'Berkeley', 'CA', '94705', 1)
('998-72-3567', 'Ringer', 'Albert', '801 826-0752', '67 Seventh Av.', 'Salt Lake City', 'UT', '84152', 1)
('899-46-2035', 'Ringer', 'Anne', '801 826-0752', '67 Seventh Av.', 'Salt Lake City', 'UT', '84152', 1)
('722-51-5454', 'DeFrance', 'Michel', '219 547-9982', '3 Balding Pl.', 'Gary', 'IN', '46403', 1)
('807-91-6654', 'Panteley', 'Sylvia', '301 946-8853', '1956 Arlington Pl.', 'Rockville', 'MD', '20853', 1)
('893-72-1158', 'McBadden', 'Heather', '707 448-4982', '301 Putnam', 'Vacaville', 'CA', '95688', 0)
('724-08-9931', 'Stringer', 'Dirk', '415 843-2991', '5420 Telegraph Av.', 'Oakland', 'CA', '94609', 0)
('274-80-9391', 'Straight', 'Dean', '415 834-2919', '5420 College Av.', 'Oakl

In [97]:
import pandas as pd

In [98]:
#read the file
df = pd.read_csv('500-us-users.csv')
df.head()

,first_name,last_name,address,city,county,state,zip,phone1,phone2
0,James,Butt,6649 N Blue Gum St,New Orleans,Orleans,LA,70116,504-621-8927,504-845-1427
1,Josephine,Darakjy,4 B Blue Ridge Blvd,Brighton,Livingston,MI,48116,810-292-9388,810-374-9840
2,Art,Venere,8 W Cerritos Ave #54,Bridgeport,Gloucester,NJ,8014,856-636-8749,856-264-4130
3,Lenna,Paprocki,639 Main St,Anchorage,Anchorage,AK,99501,907-385-4412,907-921-2010
4,Donette,Foller,34 Center St,Hamilton,Butler,OH,45011,513-570-1893,513-549-4561


In [99]:
#create a schema
cur.execute("CREATE SCHEMA People")

In [100]:
#create a table
cur.execute("""  CREATE TABLE People.Users (    
                   U_id          serial CONSTRAINT UPKCL_auidind PRIMARY KEY ,
                   fname       varchar(40)       NOT NULL,
                   lname       varchar(40)       NOT NULL,
                   address        varchar(40)           NULL,
                   city           varchar(20)           NULL,
                   county           varchar(40)           NULL,
                   state          char(2)               NULL,
                   zip            char(5)               NULL,
                   landline        char(12)      NOT NULL DEFAULT ('UNKNOWN'),
                   mobile        char(12)        NOT NULL DEFAULT ('UNKNOWN')
            );""")

In [101]:
#populating the database
for row in df.itertuples():
    cur.execute(f"""
                INSERT INTO People.Users (fname, lname, address, city, county, state, zip, landline, mobile) 
                values (\'{row.first_name}\', \'{row.last_name}\', \'{row.address}\', \'{row.city}\', \'{row.county}\', 
                \'{row.state}\', \'{row.zip}\', \'{row.phone1}\', \'{row.phone2}\')
    
    """)
con.commit() 

In [102]:
cur.execute("""
            SELECT count(fname), state 
            FROM People.Users
            GROUP BY state
            """)
print(cur.fetchall())

[(2, 'NV'), (22, 'OH'), (46, 'NY'), (1, 'AR'), (5, 'CT'), (3, 'WY'), (9, 'LA'), (1, 'MT'), (14, 'MI'), (1, 'NH'), (6, 'OR'), (1, 'DC'), (11, 'WI'), (5, 'RI'), (10, 'TN'), (8, 'WA'), (12, 'MA'), (8, 'CO'), (7, 'VA'), (9, 'AZ'), (1, 'ND'), (4, 'HI'), (9, 'IN'), (2, 'NE'), (28, 'FL'), (3, 'ME'), (1, 'SD'), (52, 'NJ'), (1, 'UT'), (3, 'SC'), (72, 'CA'), (32, 'TX'), (1, 'KY'), (2, 'NM'), (4, 'MS'), (4, 'MO'), (8, 'NC'), (1, 'OK'), (7, 'GA'), (4, 'ID'), (7, 'MN'), (29, 'PA'), (17, 'MD'), (6, 'AK'), (15, 'IL'), (1, 'IA'), (5, 'KS')]


In [104]:
query1 = """ SELECT fname, zip
        FROM People.Users
        ORDER BY zip;
        """
zip1 = pd.read_sql(query1, con)
zip1.head(20)

,fname,zip
0,Ozell,10002
1,Brock,10003
2,Mirta,10004
3,Tawna,10009
4,Jose,10011
5,Willow,10011
6,Layla,10011
7,Cyril,10013
8,Derick,10013
9,Haydee,10016
